In [ ]:
from ohbm_optim import _load_sheets, _parse_sheets, _sim_data

verbose       = False  # set to True to print extensive output info
verbose_optim = True   # set to True to print running optimization
verbose_plot  = True   # display similarity matrix
save_outputs  = True   # save symposia schedule and (if plotted) similatity matix plot

# Load and parse data
client_id = 'client_id.json'
fname = 'https://docs.google.com/spreadsheets/d/1uLQzBNWhVLEJBkFHC_WQf2h0X1YQDuIsov3EDmzWv5A'
sheets = _load_sheets(fname, client_id)
data = _parse_sheets(["symposia", "symposia_constraints", "sessions", "rooms", "symposia_similarity"])

## Similarity between symposia
sim_mat = _sim_data(data)

    
##################################################################################################
# Run optimizer:
##################################################################################################
ns             = len(panel_titles)             # number of symposia
nt             = len(timeslotsList)            # number of time slots
nr             = len(room_names)              # number of rooms

m = cp_model.CpModel()
x = {}
for s in range(ns):
    for t in range(nt):
        for r in range(nr):
            x[(s,t,r)] = m.NewBoolVar('schedule_s%it%ir%i' % (s,t,r))
for t in range(nt):
    for r in range(nr):
        m.Add(sum(x[(s,t,r)] for s in range(ns)) <= 1)
for s in range(ns):
    m.Add(sum(x[(s,t,r)] for r in range(nr) for t in range(nt)) == 1)
for t in range(nt):
    m.Add(sum(x[(s,t,r)] for s in range(ns) for r in range(nr)) == symp_per_slot[t])

print('===== constraints: =====')
for i in set_room.dropna("").index.values:
    assignRooms(i,set_room.dropna("")[i])
for i in set_session.dropna("").index.values:
    scheduleSymposia(i,inputs['sessions'][inputs['sessions'].Name.str.match(set_session.dropna("")[i])].index.values[0])
for i in set_priority.dropna("").index.values:
    prioritizeScheduling(i)

i_pairs_list = list(itertools.combinations(range(ns),2))
y = {}
for i,ip in i_pairs_list:
    for t in range(nt):
        y[(i,ip,t)] = m.NewBoolVar('overlap_i%iip%it%i' % (i,ip,t))

for i in avoid_overlap.dropna("").index.values:
    avoidSymposiaOverlap(inputs['symposia'][names.str.match(symposia1[i].replace('(','\(').replace(')','\)'))].index.values[0], 
                         inputs['symposia'][names.str.match(symposia2[i].replace('(','\(').replace(')','\)'))].index.values[0])
for i in ensure_overlap.dropna("").index.values:
    symposiaOverlap(inputs['symposia'][names.str.match(symposia1[i].replace('(','\(').replace(')','\)'))].index.values[0], 
                    inputs['symposia'][names.str.match(symposia2[i].replace('(','\(').replace(')','\)'))].index.values[0])

for i,ip in i_pairs_list:
    for t in range(nt):
        m.Add(sum(x[i,t,r] for r in range(nr)) + sum(x[ip,t,r] for r in range(nr)) - y[i,ip,t] <= 1)
        m.Add(2*y[i,ip,t] - sum(x[i,t,r] for r in range(nr)) - sum(x[ip,t,r] for r in range(nr))  <= 0)

m.Maximize(sum(sim_mat[i][ip] * y[(i,ip,t)] for i,ip in i_pairs_list for t in range(nt)))
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 8
solver.Solve(m)

# ##################################################################################################
# # Results:
# ##################################################################################################
# Statistics
print()
print('===== statistics: =====')
print('  - Stats           : %i' % solver.ObjectiveValue())
print('  - wall time       : %f s' % solver.WallTime())
print(m.ModelStats())
print()

# Schedule
newOrder = []
print()
print("===== Schedule: =====")
file1 = open("schedule.txt","w")
for t in range(nt):
    print('%s' % timeslotsList[t])
    file1.write('%s\n' % timeslotsList[t])
    for s in range(ns):        
        for r in range(nr):
            if solver.Value(x[(s,t,r)]) == 1:
                print('%s: %s' % (room_names[r],panel_titles[s]))
                newOrder.append(s)
                file1.write('%s: %s\n' % (room_names[r],panel_titles[s].encode('utf8').decode("utf-8")))               
    print()
    file1.write('\n')
file1.close()

%matplotlib inline
if verbose_plot:
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm

    panel_titles_reordered = [panel_titles[i] for i in newOrder]

    fig = plt.figure(figsize=(12,12))
    plt.imshow(sim_mat[:,newOrder][newOrder].squeeze(), interpolation='nearest', cmap=cm.jet_r, vmin=.95*10000.)
    ax = plt.gca()
    ax.set_yticks(np.arange(0, len(panel_titles_reordered), 1))
    ax.set_xticks(np.arange(0, len(panel_titles_reordered), 1))
    ax.set_yticklabels(panel_titles_reordered)
    ax.set_xticklabels(panel_titles_reordered)
    plt.xticks(rotation=90)

    for i in np.cumsum(np.asarray(symp_per_slot))-0.5:
        plt.axvline(x=i, color='white', linewidth=6)
        plt.axhline(y=i, color='white', linewidth=6)

    if save_outputs:
        plt.savefig('symposia_similarity.png')

    plt.show()

    # squares along diagonal reflext similarity of symposia sessions:
